In [ ]:
pip install pandas-datareader

In [1]:
# Import Dependencies
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import os
from datetime import datetime
from MCForecastTools import MCSimulation
import requests
import pandas_datareader as pdr
%matplotlib inline

In [2]:
load_dotenv()


True

In [3]:
alpaca_api_key = os.getenv("alpaca_api_key")
alpaca_secret_key = os.getenv("alpaca_secret_key")

alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

# Function to Validate Tickers
def validateTick(tick):
    
    # Try Inputting into Yahoo Stocks
    try:
        pdr.DataReader(f'{tick}','yahoo',date)
    
    # Raise Error if Invalid
    except:
        raise ValueError("Ticker Not Valid.")
        
# Function to Validate Date
def validate(date_text):
    
    # Try Check on Format
    try:
        datetime.strptime(date_text, '%Y-%m-%d')
        
    # Raise Error if Invalid
    except ValueError:
        raise ValueError("Incorrect data format, should be YYYY-MM-DD")

In [7]:
# Set Empty Dictionary
stock_dict = {}

# Set Empty List
stock_list = []

# Pick a Start Date
date = input("Pick a start date for your analysis (YYYY-MM-DD): ")
validate(date)

# While Loop to Pick Stocks
while (True):
    name = input("Add a stock ticker to the list: ").upper()
    validateTick(name)
    stock_dict[name] = date
    stock_list.append(name)
    cont = input("Press 'y' to Add Another Stock, press 'Enter' to exit")
    if cont == "":
        break;
    

Pick a start date for your analysis (YYYY-MM-DD):  2018-01-01
Add a stock ticker to the list:  AAPL
Press 'y' to Add Another Stock, press 'Enter' to exit 


In [8]:

# Function to Create Stock DataFrames
def stockCreation(stock_dict, stock_list):
    
    # Set Empty List
    func_list = []
    
    # Get Today's Date
    today = datetime.now().strftime("%Y-%m-%d")

    # Convert Today's Date to ISO Format 
    today = pd.Timestamp(today, tz="America/New_York").isoformat()

    # Set TimeFrame
    timeframe = "1D"

    # Create DataFrame
    portfolio = alpaca.get_barset(
        stock_list,
        timeframe,
        start = date,
        end = today
    ).df
    
    # Add Closing Prices to Empty List
    for key, value in stock_dict.items():
        globals()[key] = portfolio[key]["close"]
        func_list.append(globals()[key])
    
    # Create DataFrame out of Closing Prices
    portfolio_close = pd.concat(func_list, axis='columns', join='inner')
    
    # Rename Columns
    portfolio_close.columns = stock_list
    
    # Return Both DataFrames
    return portfolio, portfolio_close

In [9]:
portfolio = stockCreation(stock_dict, stock_list)[0]
portfolio.head()

AAPL                                     
                               open      high     low    close    volume
time                                                                    
2020-07-22 00:00:00-04:00  386.7700  391.9000  386.41  389.090  18899399
2020-07-23 00:00:00-04:00  387.9935  388.3100  369.64  371.560  25747322
2020-07-24 00:00:00-04:00  363.9500  371.8800  356.58  370.420  43883515
2020-07-27 00:00:00-04:00  374.8400  379.6200  373.92  379.215  26920001
2020-07-28 00:00:00-04:00  377.4700  378.1986  372.99  373.040  23023333

In [ ]:
portfolio_close = stockCreation(stock_dict, stock_list)[1]
portfolio_close.head()

# Exponentially Weighted Moving Average (EWMA)

### Definition: A view of the average price (open, close, mean) of a stock that gives more weight to the more recent values of the asset.  

### Purpose: The EWMA of a stock can be used as a condfidence indiacator, meaning that a strong but declining stock should have a hard time moving below the EWMA.EWMA can help you to determine what actions to take with an asset depending on its relative position to the close price. 

In [ ]:
portfolio_close.head()

In [ ]:
df = portfolio_close

def ewmas(df, win, keepSource):

    df_temp = df.copy()

    #change names of columns
    columnNames = list(df_temp.columns.values).copy()
    removeNames = columnNames.copy()

    i = 0
    for col in columnNames:

        # Make new names for ewmas
        ewmaName = columnNames[i] + '_EWMA' + str()   

        # Add ewmas
        df_temp[ewmaName] = df[columnNames[i]].ewm(halflife=21).std()

        i = i + 1

    # Remove estimates with insufficient window length
    df_temp = df_temp.iloc[win:]

    # Remove or keep source data
    if keepSource == False:
        df_temp = df_temp.drop(removeNames,1)

    return df_temp

df_new = ewmas(df = df, win = 2, keepSource = True)
df_new.tail()

In [ ]:
df_new.plot(figsize=(20,10),title='Portfolio Exponentially Weighted Moving Average and Close Price');

### Using Exponentially Weighted Moving Average with overlapping closing prices, what would be your action for the assets in the portfolio you created? 

# Risk Analysis

### Definition: Looking at individual assets, risk and return can be directly correlated. Often as risk increases the opportunity for a large return will also increase, but also a large loss. Looking at an entire portfolio a more risk adverse portfolio will be diverse and have stocks or assets that do not directly correlate with each other. 

### Purpose: Risk Analysis is a good tool to weigh the risk versus return for an entire portfolio or a single asset or security. 

In [ ]:
#variables needed
#stocks = portfolio_returns
#weights
weights = np.full((1,len(portfolio_close.columns)), 1/len(portfolio_close.columns))[0].tolist()

#calculate the daily returns of our portfolio
portfolio_returns = portfolio_close.pct_change()
portfolio_returns.head(10)

In [ ]:
portfolio_returns = portfolio_returns.dropna().copy()
portfolio_returns.head()

In [ ]:
#calculate the a average return for each stock
portfolio_returns_avg_rtn = portfolio_returns.mean()
portfolio_returns_avg_rtn

In [ ]:
allocated_daily_returns = (weights * portfolio_returns_avg_rtn)

portfolio_return_pct = np.sum(allocated_daily_returns)
print(f"""

The percent return of the portfolio is {round(portfolio_return_pct*100,2)}%""")

In [ ]:
#add daily returns to our dataframe 
portfolio_returns['portfolio_daily_returns'] = portfolio_returns.dot(weights)
portfolio_returns.head()

In [ ]:
#calculate cumulative returns
Cumulative_returns_daily = (1+portfolio_returns).cumprod()
Cumulative_returns_daily.head()

In [ ]:
#plot cumulative returns for portfolio and individual stocks
Cumulative_returns_daily.plot();

In [ ]:
Cumulative_returns_daily['portfolio_daily_returns'].sum()

In [ ]:
#calculate the covariance of the portfolio for risk analysis
#a positive covariance means the assets move in the same direction
#a negative covariance means the assets move in different directions and ultimately lowers your risk through diversivication

covariance_portfolio = portfolio_returns.iloc[:,:-1]
covariance_portfolio = (covariance_portfolio.cov())*252

covariance_portfolio

In [ ]:
#calculate variance and risk

portfolio_variance = np.dot(weights,np.dot(covariance_portfolio, weights))

#standard deviation (risk of portfolio)
portfolio_risk = np.sqrt(portfolio_variance)
portfolio_risk


In [ ]:
print(f"""This portfolio has a return of {round(Cumulative_returns_daily['portfolio_daily_returns'].sum() - 100,2)}% 
and a risk of {round(portfolio_risk *100, 2)}%.""")

### Knowing the risk and return of this portfolio, how would you optimize it or change it to reduce risk or increase return? 